In [ ]:
import numpy as np;
import matplotlib.pyplot as plt;
import pyaudio;
import wave;
import scipy.io.wavfile as sciiowav;
import scipy.signal as scisig;

%matplotlib notebook
cmap_name = 'jet';

In [ ]:
A4 = 440.0; #en Hz

def num2freq(n):
    return A4*np.power(2.0,(n-69.0)/12.0);

def num2note(n):
    return names[(120+n)%12]+ str(-1+(n/12));

names = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B'];    


print num2note(60) + ' ~ ' + str(num2freq(60)) + ' Hz';
print num2note(69) + ' ~ ' + str(num2freq(69)) + ' Hz';

In [ ]:
def get_spectrogram(x,win,step,rate):
    win_size   = win.shape[0];
    win_size_2 = win.shape[0]/2;

    espectro_t = np.zeros((0,));
    espectro_x = np.zeros((win_size_2,0));
    
    j = 0;
    for i in range(0,x.shape[0],step):
        espectro_t = np.r_[espectro_t,i/rate];
        aux_x = np.zeros((0,));
        if j % 100 == 0:
            print i/rate;
        aux_x = np.r_[np.zeros((max(win_size_2-i,0),)),
                      x[max(0,i-win_size_2):min(x.shape[0],i+win_size_2)],
                      np.zeros((max(i+win_size_2-x.shape[0],0),))];
        aux_abs = np.abs(np.fft.rfft(win*aux_x))[:-1];
        espectro_x = np.c_[espectro_x,aux_abs];
        j = j + 1;
    return espectro_t,espectro_x/(np.max(espectro_x)+1e-20);

In [ ]:
SONG = "canciones/Heart_of_Gold.wav";

[RATE,SONG_F] = sciiowav.read(SONG);

RATE = 1.0*RATE;

MAX_LEN = int(40.0*RATE);
MAX_SEC = MAX_LEN/RATE;

SCALE = 4;


MONO_F = 0.5*SONG_F[:MAX_LEN:SCALE,0]+0.5*SONG_F[:MAX_LEN:SCALE,1];

RATE = RATE/SCALE;

print SONG + u'\nDuración ~ ' + str(MAX_SEC) + 's con taza ~ ' + str(RATE) + ' Hz';

time = np.linspace(0.0,MAX_SEC,MONO_F.shape[0]);


fig, ax = plt.subplots(1, 1,figsize=(12.8,9.6));
ax.plot(time,MONO_F);

In [ ]:
WIN_SIZE = 8192;
WIN_SIZE_2 = WIN_SIZE/32;
WIN =  scisig.windows.blackman(WIN_SIZE);
print "Duración de la ventana Blackman ~ " + str(WIN_SIZE/RATE) + "s con salto ~ " + str(WIN_SIZE_2/RATE)  + "s";

frec = np.linspace(0.0,0.5*RATE,WIN_SIZE_2);

STEP_SIZE = WIN_SIZE_2;
espectro_t,espectro_sonido = get_spectrogram(MONO_F,WIN,STEP_SIZE,RATE);

In [ ]:
Ctidx = int(num2freq(84)*WIN_SIZE/RATE+1);

fig,ax = plt.subplots(1,1,figsize=(12.8,9.6),sharex=True);
fig.suptitle("Espectrograma de " + SONG,fontsize=16);

ax.set_ylabel(u'Frecuencia (Hz)');
ax.set_yticks(np.arange(0,Ctidx,64));
ax.set_yticklabels(np.arange(0,Ctidx,64)*RATE/WIN_SIZE);
ax.set_xlabel(u'Tiempo (s)');
ax.set_xticks(np.arange(0,espectro_t.shape[0],256));
ax.set_xticklabels(espectro_t[::256]);


ax.imshow(espectro_sonido[:Ctidx,:],cmap=cmap_name,aspect='auto');

In [ ]:
def get_chroma_vector(X):
    C = np.zeros((12,));
    for n in range(12):
        for o in range(3,6):
            for h in range(1,6):
                k_r = h*num2freq(n+12*o)*WIN_SIZE/RATE;
                k_l = int(np.floor(k_r));
                k_h = int(np.ceil(k_l));
                u = (k_h-k_l);
                C[n] += (u*X[k_l]+(1.0-u)*X[k_h])/h;
    return C/3.0;

def get_chromagram(sx):
    cx = np.zeros((12,sx.shape[1]));
    for t in range(sx.shape[1]):
        cx[:,t] = get_chroma_vector(sx[:,t]);
    return cx;

def get_smooth(cx,L):
    smx = np.zeros((cx.shape[0],0));
    win = scisig.windows.nuttall(2*L+1);
    
    j = 0;
    for i in range(0,cx.shape[1]):
        aux_x = np.zeros((12,0));
        if j % 100 == 0:
            print i;
        aux_x = np.c_[np.zeros((cx.shape[0],max(L-i,0))),
                      cx[:,max(0,i-L):min(cx.shape[1],i+L+1)],
                      np.zeros((cx.shape[0],max(i+L+1-cx.shape[1],0)))];
        aux_abs = np.dot(aux_x,win);
        smx = np.c_[smx,aux_abs];
        j = j + 1;
        
    return smx;

c_sonido = get_chromagram(espectro_sonido);
sc_sonido = get_smooth(c_sonido,2);

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12.8,9.6),sharex=True);
fig.suptitle("Chromagrama de " + SONG,fontsize=16);

ax.set_ylabel(u'Nota');
ax.set_yticks(np.arange(12));
ax.set_yticklabels(['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']);
ax.set_xlabel(u'Tiempo (s)');
ax.set_xticks(np.arange(0,espectro_t.shape[0],256));
ax.set_xticklabels(espectro_t[::256]);

ax.imshow(sc_sonido,cmap=cmap_name,aspect='auto');

In [ ]:
#ACORDES
#                      C, C#,  D, D#,  E,  F, F#,  G, G#,  A, A#,  B

#                      0   1   1   0   1   1   0   1   1   0   1   1  dim7  --- no

#                      0   1   1   1   0   1   0   1   1   1   1   1  b5    --- no
#                      0   1   1   1   0   1   1   1   0   1   1   1  #5    --- no

acordes = np.array([ #[ 0,  1,  1,  1,  1,  1,  1,  0,  1,  1,  1,  1], #5
                     [ 0,  1,  1,  1,  0,  1,  1,  0,  1,  1,  1,  1], #M
                     [ 0,  1,  1,  0,  1,  1,  1,  0,  1,  1,  1,  1], #m
                     #[ 0,  1,  1,  0,  1,  1,  0,  1,  1,  1,  1,  1], #dim
                     [ 0,  1,  0,  1,  1,  1,  1,  0,  1,  1,  1,  1], #sus2
                     [ 0,  1,  1,  1,  0,  1,  1,  0,  1,  1,  1,  1] #sus4
                     #[ 0,  1,  1,  1,  0,  1,  1,  1,  0,  1,  1,  1], #aug
                     #[ 0,  1,  1,  0,  1,  1,  1,  0,  1,  1,  1,  0], #mmaj7
                     #[ 0,  1,  1,  1,  0,  1,  1,  0,  1,  1,  1,  0], #Mmaj7
                     #[ 0,  1,  1,  0,  1,  1,  1,  0,  1,  1,  0,  1], #m7
                     #[ 0,  1,  1,  1,  0,  1,  1,  0,  1,  1,  0,  1] #M7
                     ]);

acordes_suf = [#'5',
               'M',
               'm',
               #'dim',
               'sus2',
               'sus4',
               #'aug',
               #'mmaj7',
               #'Mmaj7',
               #'m7',
               #'M7'
              ];

def get_acordes(n):
    return np.roll(acordes,n);

notes_d = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B'];
ACORDES   = np.zeros((0,12));
ACORDES_N = [];
for n in range(12):
    ACORDES = np.r_[ACORDES,get_acordes(n)];
    ACORDES_N = ACORDES_N + [notes_d[n]+s for s in acordes_suf];
ACORDES_S = np.sum(ACORDES,axis=1);
print len(ACORDES_N)

In [ ]:
def get_dot(X):
    aux = 1e-20+np.sqrt(np.dot(ACORDES,np.power(X,2.0)))/ACORDES_S;
    aux /= np.sum(aux);
    return aux;
t = 50;
print str(espectro_t[t]) + 's';
dotCgrama =  get_dot(c_sonido[:,t]);
argCgrama = np.argsort(dotCgrama);
for a in range(3):
    print ACORDES_N[argCgrama[a]] + ' ~ ' + str(dotCgrama[argCgrama[a]]);

In [ ]:
acordes_sonido = np.zeros((len(ACORDES_N),0));
for t in range(c_sonido.shape[1]):
    acordes_sonido = np.c_[acordes_sonido,get_dot(c_sonido[:,t])];
    
a_sonido = get_smooth(acordes_sonido,5);

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12.8,9.6));
ax.set_xlabel(u'Tiempo (s)');
ax.set_xticks(np.arange(0,espectro_t.shape[0],256));
ax.set_xticklabels(espectro_t[::256]);
ax.set_ylabel(u'Acordes'); 
ax.set_yticks(np.arange(len(ACORDES_N)));
ax.set_yticklabels(ACORDES_N);
ax.imshow(a_sonido,cmap=cmap_name,aspect='auto');

In [ ]:
u_sonido = np.zeros(a_sonido.shape);
for t in range(a_sonido.shape[1]):
    aux = np.argsort(a_sonido[:,t]);
    for a in range(3):
        u_sonido[aux[a],t] = 3-a;

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12.8,9.6));
fig.suptitle("Acordes Seleccionados",fontsize=16);
ax.set_xlabel(u'Tiempo (s)');
ax.set_xticks(np.arange(0,espectro_t.shape[0],512));
ax.set_xticklabels(espectro_t[::128]);
ax.set_ylabel(u'Acordes'); 
ax.set_yticks(np.arange(len(ACORDES_N)));
ax.set_yticklabels(ACORDES_N);
ax.imshow(u_sonido,cmap='gray',aspect='auto');

In [ ]:

CHUNK = 1024;

wf = wave.open(SONG, 'rb');

samp_width = wf.getsampwidth();
nchannels  = wf.getnchannels();
rate       = wf.getframerate();
print samp_width,nchannels,rate;

p = pyaudio.PyAudio();

stream = p.open(format=p.get_format_from_width(samp_width),
                channels=nchannels,
                rate=rate,
                output=True)

data = wf.readframes(CHUNK);
while data != '':
    stream.write(data)
    data = wf.readframes(CHUNK);

stream.stop_stream()
stream.close()

p.terminate()